In [ ]:
# | default_exp _components.aiokafka_loop

In [ ]:
# | export

from typing import *

from os import environ
import asyncio
from contextlib import asynccontextmanager, contextmanager

import aiokafka

from fast_kafka_api.confluent_kafka import create_testing_topic
from fast_kafka_api.logger import get_logger

In [ ]:
from fast_kafka_api.testing import true_after

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
logger = get_logger(__name__, level=20)
logger.debug("ok")

In [ ]:
kafka_server_url = environ["KAFKA_HOSTNAME"]
kafka_server_port = environ["KAFKA_PORT"]

kafka_config = {
    "bootstrap.servers": f"{kafka_server_url}:{kafka_server_port}",
    "group.id": f"{kafka_server_url}:{kafka_server_port}_group",  # ToDo: Figure out msg deletion from kafka after consuming once
    "auto.offset.reset": "earliest",
}

In [ ]:
# | export


@asynccontextmanager
async def create_and_produce_testing_topic(
    msgs: List[bytes], kafka_config: Dict[str, str] = kafka_config, *, seed: int = 42
) -> str:

    with create_testing_topic(kafka_config, "my_topic_", seed=seed) as (topic, _, _):

        producer = aiokafka.AIOKafkaProducer(
            bootstrap_servers=kafka_config["bootstrap.servers"]
        )
        logger.info(f"Producer {producer} created.")

        await producer.start()
        logger.info(f"Producer {producer} stared.")
        try:
            fx = [
                producer.send_and_wait(topic, msg, key=f"{i % 17}".encode("utf-8"), )
                for i, msg in enumerate(msgs)
            ]
            await producer.flush()
            sent_msgs = [await f for f in fx]
            logger.info(f"Sent messages: {len(sent_msgs)=}")

            yield topic
        finally:
            await producer.stop()
            logger.info(f"Producer {producer} stoped.")

In [ ]:
async def test_create_and_produce_testing_topic(
    *,
    kafka_config: Dict[str, str] = kafka_config,
    is_shutting_down_f: Optional[Callable[[], bool]] = None,
    seed: int = 42,
):
    msgs_sent = 317
    msgs = [f"Hello world {i:05d}".encode("utf-8") for i in range(msgs_sent)]
    
    async with create_and_produce_testing_topic(msgs, seed=seed) as topic:
        consumer = aiokafka.AIOKafkaConsumer(
            topic,
            bootstrap_servers=kafka_config["bootstrap.servers"],
            auto_offset_reset="earliest",
            max_poll_records=100,
        )
        logger.info(f"Consumer {consumer} created.")
        await consumer.start()
        logger.info(f"Consumer {consumer} started.")
        msgs_received = 0
        try:
            while True:
                msgs = await consumer.getmany(timeout_ms=100)
                for k, v in msgs.items():
                    msgs_received = msgs_received + len(v)
                    if len(v) > 0:
                        print(f"total received: {msgs_received}")
#                     for msg in v:
#                         print(
#                             "topic={}   partition{:03d}   offset={:05d}   key={:5s}   value={} timestamp_ms={}".format(
#                                 msg.topic,
#                                 msg.partition,
#                                 msg.offset,
#                                 msg.key.decode("utf-8"),
#                                 msg.value,
#                                 msg.timestamp,
#                             )
#                         )
                if is_shutting_down_f and is_shutting_down_f():
                    break
                    
        finally:
            assert msgs_received == msgs_sent
            print(f"Total messages received: {msgs_received}")
            await consumer.stop()
            logger.info(f"Consumer {consumer} stopped.")


# asyncio.run(test_create_and_produce_testing_topic())
await test_create_and_produce_testing_topic(is_shutting_down_f=true_after(5), seed=3)

[INFO] fast_kafka_api.confluent_kafka: create_missing_topics(['my_topic_vcegeuwpbc']): new_topics = [NewTopic(topic=my_topic_vcegeuwpbc,num_partitions=3)]


%4|1670943129.522|CONFWARN|rdkafka#producer-10| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1670943129.522|CONFWARN|rdkafka#producer-10| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance


[INFO] __main__: Producer <aiokafka.producer.producer.AIOKafkaProducer object> created.
[INFO] __main__: Producer <aiokafka.producer.producer.AIOKafkaProducer object> stared.


%4|1670943130.642|CONFWARN|rdkafka#producer-12| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1670943130.642|CONFWARN|rdkafka#producer-12| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance


[INFO] __main__: Sent messages: len(sent_msgs)=317
[INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'my_topic_vcegeuwpbc'})
[INFO] __main__: Consumer <aiokafka.consumer.consumer.AIOKafkaConsumer object> created.
[INFO] aiokafka.consumer.group_coordinator: Metadata for topic has changed from {} to {'my_topic_vcegeuwpbc': 3}. 
[INFO] __main__: Consumer <aiokafka.consumer.consumer.AIOKafkaConsumer object> started.
total received: 100
total received: 112
total received: 212
total received: 223
total received: 317
Total messages received: 317
[INFO] __main__: Consumer <aiokafka.consumer.consumer.AIOKafkaConsumer object> stopped.
[INFO] __main__: Producer <aiokafka.producer.producer.AIOKafkaProducer object> stoped.
[INFO] fast_kafka_api.confluent_kafka: Topic 'my_topic_vcegeuwpbc' deleted.
